In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
#access the wikipedia table
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
r=requests.get(url).text
soup = BeautifulSoup(r,'html')
table=soup.find_all('table')
df=pd.read_html(str(table))[0]

In [3]:
#The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
df1=df.rename(columns={"Postal Code":"PostalCode","Neighbourhood":"Neighborhood"})

In [4]:
#Ignore cells with a borough that is 'Not assigned'
df2=df1[df.Borough != "Not assigned"]
df3=df2.reset_index()

In [5]:
#Combine neighborhoods based on post code
df4=df3.groupby("PostalCode")["Neighborhood"].agg([("Neighborhood", ','.join)]).reset_index()

In [6]:
#add "Borough" column
df4["Borough"]=df3["Borough"]

In [7]:
#arrange columns order
df5=df4[["PostalCode","Borough","Neighborhood"]]

In [8]:
#if a Borough cell has a cell but a "Not assigned" Neighborhood to make Neighborhood as as Borough
#check "Not assigned" Neighborhood
df5.loc[df5["Neighborhood"] == "Not assigned"]
#since no output which means no "Not assigned" Neighborhood exists

,PostalCode,Borough,Neighborhood


In [9]:
#display the table
df5

,PostalCode,Borough,Neighborhood
0,M1B,North York,"Malvern, Rouge"
1,M1C,North York,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Downtown Toronto,"Guildwood, Morningside, West Hill"
3,M1G,North York,Woburn
4,M1H,Downtown Toronto,Cedarbrae
...,...,...,...
98,M9N,Etobicoke,Weston
99,M9P,Downtown Toronto,Westmount
100,M9R,East Toronto,"Kingsview Village, St. Phillips, Martin Grove ..."
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest..."


In [10]:
#print the number of rows using .shape
df5.shape

(103, 3)